In [10]:
import torch
import pandas as pd
import numpy as np
from tqdm import tqdm
import wandb

In [11]:
wandb.init(entity="ameyar3103-iiit-hyderabad",project="recurrent_conv_art", config={
    "epochs": 20,
    "batch_size": 64,
    "learning_rate": 0.001,
    "model": "RecurrentCNN"
})

## Data loading

In [12]:
df_train = pd.read_csv('wikiart_csv/style_train.csv',header=None, names=["image_path", "style_id"])
df_val = pd.read_csv('wikiart_csv/style_val.csv',header=None, names=["image_path", "style_id"])

In [13]:
# get the number of classes
num_classes = 27 # from style_class.txt


In [14]:
# Gather input data
train_images = df_train['image_path'].values
train_labels = df_train['style_id'].values

val_images = df_val['image_path'].values
val_labels = df_val['style_id'].values

In [15]:
from torchvision import transforms
import cv2

## Preprocess data and create test and train dataset

In [16]:
# create test and train dataset for dataloader

def get_image(image_path,image_size=224):
    try:
        img = cv2.imread('./wikiart/' + image_path)
        if img is None:
            raise ValueError(f"Image not loaded: ./wikiart/{image_path}")
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        h, w, _ = img.shape
        scale = 256 / min(h, w)
        new_w = int(w * scale)
        new_h = int(h * scale)
        img_resized = cv2.resize(img, (new_w, new_h))
        start_x = (new_w - image_size) // 2
        start_y = (new_h - image_size) // 2
        img_cropped = img_resized[start_y:start_y+image_size, start_x:start_x+image_size]
        img_cropped = img_cropped.astype(np.float32) / 255.0
        img_tensor = torch.from_numpy(img_cropped).permute(2, 0, 1)
        mean = torch.tensor([0.485, 0.456, 0.406]).view(3, 1, 1)
        std  = torch.tensor([0.229, 0.224, 0.225]).view(3, 1, 1)
        img_tensor = (img_tensor - mean) / std
        return img_tensor
    except Exception as e:
        print(f"Error processing {image_path}: {e}")
        return torch.zeros(3, image_size, image_size)

class WikiArtDataset(torch.utils.data.Dataset):
    def __init__(self, images, labels):
        self.images = images
        self.labels = labels

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        # image_vectors = []
        # for image in self.images:
        #     image_emb = get_image(image)
        #     image_vectors.append(image_emb)
        # image = torch.stack(image_vectors)
        image = self.images[idx]
        # label should be a one-hot encoded vector
        label = torch.zeros(num_classes)
        label[self.labels[idx]] = 1

        return image, label

train_dataset = WikiArtDataset(train_images, train_labels)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=64, shuffle=True)
val_dataset = WikiArtDataset(val_images, val_labels)
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=64, shuffle=False)

for i, (images, labels) in enumerate(train_loader):
    print(images)
    print(labels)
    break

('Realism/ivan-shishkin_dark-forest-1890.jpg', 'Post_Impressionism/paul-cezanne_portrait-of-madame-cezanne-in-a-red-dress.jpg', 'Post_Impressionism/maurice-prendergast_blue-mountains.jpg', 'Expressionism/m.c.-escher_not_detected_204655.jpg', 'Realism/edouard-manet_head-of-jean-baptiste-faure.jpg', 'Art_Nouveau_Modern/alexandre-benois_italian-comedy(1).jpg', 'Realism/vincent-van-gogh_sien-with-child-on-her-lap-1882.jpg', 'Impressionism/claude-monet_the-seine-at-lavacourt-1880.jpg', 'Rococo/antoine-watteau_italian-comedians.jpg', 'Symbolism/william-blake_the-goblin-1820.jpg', 'Expressionism/amedeo-modigliani_young-brunette-girl-sitting-1918.jpg', 'Early_Renaissance/masaccio_san-giovenale-triptych-left-panel.jpg', 'Fauvism/joan-miro_portrait-of-juanita-obrador.jpg', 'Symbolism/nicholas-roerich_himalayas-35.jpg', 'Post_Impressionism/spyros-papaloukas_at-mount-athos.jpg', 'Baroque/rembrandt_peter-and-john-at-the-gate-of-the-temple-1629.jpg', 'Symbolism/kuzma-petrov-vodkin_mother-1913.jpg', 

In [ ]:
# CNN model

import torch.nn as nn
import torch.nn.functional as F

class RecurrentCNN(nn.Module):
    def __init__(self, num_classes, lstm_hidden_size=256, dropout_prob=0.5):
        super(RecurrentCNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=1)
        self.pool1 = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)
        self.pool2 = nn.MaxPool2d(2, 2)
        self.adaptive_pool = nn.AdaptiveAvgPool2d((14, 56))
        self.lstm_input_size = 64 * 56
        self.lstm_hidden_size = lstm_hidden_size
        self.lstm = nn.LSTM(input_size=self.lstm_input_size, hidden_size=lstm_hidden_size,
                            batch_first=True, bidirectional=True)
        self.dropout = nn.Dropout(dropout_prob)
        self.fc = nn.Linear(2 * lstm_hidden_size, num_classes)

    def forward(self, x):
        x = F.relu(self.conv1(x))     
        x = self.pool1(x)              
        x = F.relu(self.conv2(x))
        x = self.pool2(x)              
        x = self.adaptive_pool(x)      
        x = x.permute(0, 2, 1, 3).contiguous()  
        batch_size, seq_len, channels, width = x.shape  
        x = x.view(batch_size, seq_len, channels * width)  
        lstm_out, _ = self.lstm(x)  
        x = lstm_out.mean(dim=1)    
        x = self.dropout(x)
        x = self.fc(x)              
        return x
    
model = RecurrentCNN(num_classes)
model.to('cuda')

# Loss and optimizer
import torch.optim as optim

wandb.watch(model, log="all")
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

## Training the model

In [ ]:
# Train the model
num_epochs = 20

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    train_bar = tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs}")
    for image_paths, labels in train_bar:
        image_tensors = torch.stack([get_image(image_path) for image_path in image_paths])
        images = image_tensors.to('cuda')
        labels = labels.to('cuda')
        
        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        train_bar.set_postfix(loss=loss.item())
    
    avg_train_loss = running_loss / len(train_loader)
    wandb.log({"epoch": epoch+1, "train_loss": avg_train_loss})
    
    # Validation Loop
    model.eval()
    val_loss = 0.0
    correct = 0
    total = 0
    with torch.no_grad():
        val_bar = tqdm(val_loader, desc="Validation")
        for image_paths, labels in val_bar:
            image_tensors = torch.stack([get_image(image_path) for image_path in image_paths])
            image_tensors = image_tensors.to('cuda')
            labels = labels.to('cuda')
            outputs = model(image_tensors)
            loss = criterion(outputs, labels)
            val_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels.argmax(dim=1)).sum().item()
            val_bar.set_postfix(loss=loss.item())
    
    avg_val_loss = val_loss / len(val_loader)
    val_accuracy = 100 * correct / total
    wandb.log({"val_loss": avg_val_loss, "val_accuracy": val_accuracy})
    print(f"Epoch {epoch+1}/{num_epochs} - Train Loss: {avg_train_loss:.4f}, Val Loss: {avg_val_loss:.4f}, Val Accuracy: {val_accuracy:.2f}%")
    if(epoch%5==0):
        torch.save(model.state_dict(), f"recurrent_cnn_epoch_{epoch}_style.pth")
        torch.save(optimizer.state_dict(), f"recurrent_cnn_optimizer_epoch_{epoch}_style.pth")

Validation: 100%|██████████| 382/382 [05:41<00:00,  1.12it/s, loss=3.72]


Epoch 1/20 - Train Loss: 2.5116, Val Loss: 2.3391, Val Accuracy: 26.43%


Validation: 100%|██████████| 382/382 [05:39<00:00,  1.13it/s, loss=2.83]


Epoch 2/20 - Train Loss: 2.2795, Val Loss: 2.2328, Val Accuracy: 29.52%


Validation: 100%|██████████| 382/382 [05:39<00:00,  1.12it/s, loss=3.09]


...

Epoch 9/20:  34%|███▍      | 303/892 [04:59<09:42,  1.01it/s, loss=1.16] 


KeyboardInterrupt: 

Error in callback <bound method _WandbInit._pause_backend of <wandb.sdk.wandb_init._WandbInit object at 0x7f6230a66500>> (for post_run_cell):


BrokenPipeError: [Errno 32] Broken pipe

: 